In [4]:
# Dependencies and Setup
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

from config import username, password, host, database

In [5]:
# File path
state_rank_filepath = '../output_file/state_health_rankings.csv'

# Read in csv file
state_rankings_df = pd.read_csv(state_rank_filepath)

# Display dataframe
state_rankings_df.head()

,year,measure_name,state,rank,value
0,2009,All Determinants,Alaska,34.0,-0.121
1,2009,All Determinants,Arizona,27.0,-0.037
2,2009,All Determinants,Arkansas,41.0,-0.264
3,2009,All Determinants,California,24.0,0.134
4,2009,All Determinants,Alabama,43.0,-0.306


In [13]:
state_rankings_df.index.name= 'id'
state_rankings_df

,year,measure_name,state,rank,value
id,,,,,
0,2009,All Determinants,Alaska,34.0,-0.121
1,2009,All Determinants,Arizona,27.0,-0.037
2,2009,All Determinants,Arkansas,41.0,-0.264
3,2009,All Determinants,California,24.0,0.134
4,2009,All Determinants,Alabama,43.0,-0.306
...,...,...,...,...,...
2269,2019,Primary Care Physicians,Vermont,11.0,188.700
2270,2019,Primary Care Physicians,Washington,23.0,150.000
2271,2019,Primary Care Physicians,Wisconsin,21.0,152.400


In [20]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [21]:
# Create table schema
class StateHealth(Base):
    __tablename__ = 'state_health_rankings'
    __table_args__ = {'extend_existing': True}
    id = Column(Integer,primary_key=True)
    year = Column(Integer)
    measure_name = Column(String(255))
    state = Column(String(255))
    rank = Column(Integer)
    value = Column(Float)

In [22]:
# Create Engine
connect_string = f'postgresql://{username}:{password}@{host}/{database}'
engine = create_engine(connect_string)

In [23]:
# Specify table
state_rankings_table = [Base.metadata.tables['state_health_rankings']]

# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine, tables=state_rankings_table)

In [24]:
# Store data to SQL database
state_rankings_df.to_sql(name='state_health_rankings', con=engine, if_exists='replace', index=True)

In [25]:
pd.read_sql_query('select * from state_health_rankings', con=engine).head()

,id,year,measure_name,state,rank,value
0,0,2009,All Determinants,Alaska,34.0,-0.121
1,1,2009,All Determinants,Arizona,27.0,-0.037
2,2,2009,All Determinants,Arkansas,41.0,-0.264
3,3,2009,All Determinants,California,24.0,0.134
4,4,2009,All Determinants,Alabama,43.0,-0.306
